# Basic Example Notebook
Hello and welcome to `PauliPropagation.jl` - a Julia package for Pauli propagation simulation of quantum circuits and quantum systems.

That is, we estimate quantities like
$ Tr[\rho \ \mathcal{E}(\hat{O})] $
where $\rho$ is an initial state, $\mathcal{E}$ is a quantum channel that is not necessarily unitary, and $\hat{O}$ is an observable preferrably sparse in Pauli basis.

In the following we are going to introduce the basic concepts and datatypes of the package.

In [1]:
using PauliPropagation

An important note on the ordering of the paulis:

In [2]:
# Pauli strings are indexed from the left
pstr = PauliString(2, [:I, :Z], [1, 2])

println("The first pauli is represented as ", getpauli(pstr, 1))
println("The second pauli is represented as ", getpauli(pstr, 2))

# When we map them to integer, they are also indexed from the left. 
# This is different from a normal binary representation, where the least significant bit is on the right.
pstr_int = getpauli(pstr, 1) * 4^0 + getpauli(pstr, 2) * 4^1
println("Integer representation of $pstr is ", pstr_int)
println("Consistency check: ", pstr_int == pstr.term)

# Reverse the pauli ordering
pstr = PauliString(2, [:Z, :I], [1, 2])
println("Integer representation of $pstr is ", pstr.term)

The first pauli is represented as 0
The second pauli is represented as 3
Integer representation of PauliString(nqubits: 2, 1.0 * IZ) is 12
Consistency check: true
Integer representation of PauliString(nqubits: 2, 1.0 * ZI) is 3


Define the number of qubits in the simulation.

In [3]:
nq = 64

64

Now define the observable $\hat{O} = Z_{32} = I_1 \otimes ... \otimes I_{31} \otimes Z_{32} \otimes I_{33}... \otimes I_{62}$.

The `PauliString` type is our high-level way of handling Pauli strings. This above constructor defines a `PauliString` that has the only non-Identity Pauli at position 32.

This library uses encodes Paulis into the bits of Integers for performance reasons, and also actions of gates onto Paulis are defined as bit operations. You can definitely get by just sticking to the high level interface. Check out some other example notebooks for more involved code.

In [4]:
pstr = PauliString(nq, [:Z, :Z], [32, 33])

PauliString(nqubits: 64, 1.0 * IIIIIIIIIIIIIIIIIIII...)

Now we specify the circuit that we want to run. We provide some simple constructors for common circuit ansätze. Here we use a generic circuit ansatz called the `hardwareefficientcircuit`. It consists of repeated RX- RZ - RX Pauli rotation gates intertwined with RYY entangling gates.

For everything to work out, we need to specify the number of circuit layers and the entangling topology. By defailt we use the so-called 1D `bricklayertopology`, but you can customize a topology via an array or tuples like `[(1, 2), (1, 3), ...]`.

In [5]:
nl = 4
topology = bricklayertopology(nq; periodic=false)
circuit = hardwareefficientcircuit(nq, nl; topology=topology)
nparams = countparameters(circuit)

1020

This circuit has a total of 1020 Pauli rotation gates. Let us define some random vector of parameters.

In [6]:
using Random
Random.seed!(42)
thetas = randn(nparams);

Now we get to the interesting part. The propagation of the Observable `pstr` through `circuit`. In other words, we apply the circuit onto the Pauli string.

First, let us do it exactly.

In [7]:
@time psum = propagate(circuit, pstr, thetas)

  2.092321 seconds (621.08 k allocations: 209.499 MiB, 0.96% gc time, 16.41% compilation time)


PauliSum(nqubits: 64, 676155 Pauli terms:
 -3.2982e-5 * IIIIIIIIIIIIIIIIIIII...
 -2.9381e-7 * IIIIIIIIIIIIIIIIIIII...
 6.4219e-9 * IIIIIIIIIIIIIIIIIIII...
 -1.7381e-5 * IIIIIIIIIIIIIIIIIIII...
 -4.1581e-7 * IIIIIIIIIIIIIIIIIIII...
 -1.4135e-9 * IIIIIIIIIIIIIIIIIIII...
 8.9619e-5 * IIIIIIIIIIIIIIIIIIII...
 3.2047e-9 * IIIIIIIIIIIIIIIIIIII...
 -1.4298e-6 * IIIIIIIIIIIIIIIIIIII...
 1.3717e-7 * IIIIIIIIIIIIIIIIIIII...
 -2.8026e-7 * IIIIIIIIIIIIIIIIIIII...
 -2.4646e-8 * IIIIIIIIIIIIIIIIIIII...
 3.8644e-8 * IIIIIIIIIIIIIIIIIIII...
 8.1771e-7 * IIIIIIIIIIIIIIIIIIII...
 2.2464e-7 * IIIIIIIIIIIIIIIIIIII...
 8.4275e-7 * IIIIIIIIIIIIIIIIIIII...
 5.8563e-7 * IIIIIIIIIIIIIIIIIIII...
 7.9623e-9 * IIIIIIIIIIIIIIIIIIII...
 -4.7137e-7 * IIIIIIIIIIIIIIIIIIII...
 -4.5729e-7 * IIIIIIIIIIIIIIIIIIII...
  ⋮)

Just to run it again after Julia's jit-compilation:

In [8]:
@time propagate(circuit, pstr, thetas);

  1.725661 seconds (2.23 k allocations: 169.721 MiB, 1.55% gc time)


How is it possible that we can exactly run 64-qubit circuits? We leverage a concept referred to as the _entanglement lightcone_. But we don't do it manually, the Pauli propagation framework does it naturally. Still, we created over 160k Pauli strings in an object of type `PauliSum`. As the name suggests, this is a sum of Pauli strings. Be very mindful of the fact that the runtime will initially scale exponentially with the circuit depth!

The `psum` object may already be of interest of you. But here is a quick and easy way how to evaluate the expectation value if initial state is the zero-state, i.e. $\rho = |0\langle\rangle 0|$.

In [9]:
overlapwithzero(psum)

-0.023263767905311573

Remember, this was an exact calculation. Well actually, our library has a default truncation threshold of Pauli strings with coefficients smaller than 1e-10. Here is how you control the minimum absolute coefficient threshold `min_abs_coeff`.

In [10]:
@time psum_exact = propagate(circuit, pstr, thetas; min_abs_coeff = 0)

  1.772221 seconds (80.24 k allocations: 174.920 MiB, 0.29% gc time, 3.93% compilation time)


PauliSum(nqubits: 64, 692199 Pauli terms:
 -3.2982e-5 * IIIIIIIIIIIIIIIIIIII...
 -2.9381e-7 * IIIIIIIIIIIIIIIIIIII...
 6.4208e-9 * IIIIIIIIIIIIIIIIIIII...
 -1.7381e-5 * IIIIIIIIIIIIIIIIIIII...
 -4.1584e-7 * IIIIIIIIIIIIIIIIIIII...
 -1.3047e-9 * IIIIIIIIIIIIIIIIIIII...
 8.9619e-5 * IIIIIIIIIIIIIIIIIIII...
 3.2484e-9 * IIIIIIIIIIIIIIIIIIII...
 -1.4298e-6 * IIIIIIIIIIIIIIIIIIII...
 1.3717e-7 * IIIIIIIIIIIIIIIIIIII...
 -2.8026e-7 * IIIIIIIIIIIIIIIIIIII...
 -2.4637e-8 * IIIIIIIIIIIIIIIIIIII...
 3.8644e-8 * IIIIIIIIIIIIIIIIIIII...
 8.177e-7 * IIIIIIIIIIIIIIIIIIII...
 2.2464e-7 * IIIIIIIIIIIIIIIIIIII...
 8.4275e-7 * IIIIIIIIIIIIIIIIIIII...
 5.8563e-7 * IIIIIIIIIIIIIIIIIIII...
 7.9482e-9 * IIIIIIIIIIIIIIIIIIII...
 -4.7137e-7 * IIIIIIIIIIIIIIIIIIII...
 -4.5729e-7 * IIIIIIIIIIIIIIIIIIII...
  ⋮)

And the expectation value is pretty much the same.

In [11]:
print("The error with our default truncation of `1e-10` is ", abs(overlapwithzero(psum_exact) - overlapwithzero(psum)))

The error with our default truncation of `1e-10` is 2.641169470629645e-10

How about less strict truncation thresholds?

In [12]:
min_abs_coeff = 1e-3
@time psum_coeff = propagate(circuit, pstr, thetas; min_abs_coeff = min_abs_coeff)

  0.063274 seconds (61.24 k allocations: 8.588 MiB, 67.05% compilation time)


PauliSum(nqubits: 64, 4560 Pauli terms:
 0.0014334 * IIIIIIIIIIIIIIIIIIII...
 0.0032365 * IIIIIIIIIIIIIIIIIIII...
 -0.0066408 * IIIIIIIIIIIIIIIIIIII...
 -0.0015111 * IIIIIIIIIIIIIIIIIIII...
 -0.0084898 * IIIIIIIIIIIIIIIIIIII...
 -0.0012799 * IIIIIIIIIIIIIIIIIIII...
 -0.0019503 * IIIIIIIIIIIIIIIIIIII...
 -0.0010623 * IIIIIIIIIIIIIIIIIIII...
 -0.0021734 * IIIIIIIIIIIIIIIIIIII...
 0.0012124 * IIIIIIIIIIIIIIIIIIII...
 0.0010036 * IIIIIIIIIIIIIIIIIIII...
 -0.0032487 * IIIIIIIIIIIIIIIIIIII...
 0.0011675 * IIIIIIIIIIIIIIIIIIII...
 0.0020944 * IIIIIIIIIIIIIIIIIIII...
 -0.0011844 * IIIIIIIIIIIIIIIIIIII...
 -0.0018068 * IIIIIIIIIIIIIIIIIIII...
 0.052429 * IIIIIIIIIIIIIIIIIIII...
 -0.019609 * IIIIIIIIIIIIIIIIIIII...
 0.0039868 * IIIIIIIIIIIIIIIIIIII...
 -0.038348 * IIIIIIIIIIIIIIIIIIII...
  ⋮)

In [13]:
print("The error with `min_abs_coeff = $min_abs_coeff` is ", abs(overlapwithzero(psum_exact) - overlapwithzero(psum_coeff)))

The error with `min_abs_coeff = 0.001` is 0.0019928751244866864

The expectation value is still surprisingly precise.

Another very general but powerful truncation is one based on _Pauli weight_. We can truncate Pauli strings that have many non-Identity Paulis, which has been proven to be a valid truncation for random circuits, but it also works well in practice. We can pass this as the keyword argument `max_weight`.

In [14]:
max_weight = 5
@time psum_weight = propagate(circuit, pstr, thetas; max_weight = max_weight)

  0.183364 seconds (81.45 k allocations: 14.864 MiB, 29.62% compilation time)


PauliSum(nqubits: 64, 29497 Pauli terms:
 1.0337e-8 * IIIIIIIIIIIIIIIIIIII...
 6.8938e-6 * IIIIIIIIIIIIIIIIIIII...
 9.6722e-5 * IIIIIIIIIIIIIIIIIIII...
 -6.5883e-10 * IIIIIIIIIIIIIIIIIIII...
 -1.1134e-6 * IIIIIIIIIIIIIIIIIIII...
 1.7248e-5 * IIIIIIIIIIIIIIIIIIII...
 2.682e-7 * IIIIIIIIIIIIIIIIIIII...
 1.8743e-6 * IIIIIIIIIIIIIIIIIIII...
 9.097e-5 * IIIIIIIIIIIIIIIIIIII...
 0.0015115 * IIIIIIIIIIIIIIIIIIII...
 1.6867e-5 * IIIIIIIIIIIIIIIIIIII...
 -7.4469e-8 * IIIIIIIIIIIIIIIIIIII...
 1.7503e-5 * IIIIIIIIIIIIIIIIIIII...
 -6.5277e-6 * IIIIIIIIIIIIIIIIIIII...
 1.8662e-8 * IIIIIIIIIIIIIIIIIIII...
 -1.6903e-5 * IIIIIIIIIIIIIIIIIIII...
 -3.9027e-7 * IIIIIIIIIIIIIIIIIIII...
 6.7058e-9 * IIIIIIIIIIIIIIIIIIII...
 -4.975e-7 * IIIIIIIIIIIIIIIIIIII...
 -1.2984e-5 * IIIIIIIIIIIIIIIIIIII...
  ⋮)

In [15]:
print("The error with `max_weight = $max_weight` is ", abs(overlapwithzero(psum_exact) - overlapwithzero(psum_weight)))

The error with `max_weight = 5` is 0.010907001667522679

One mindset to adopt when using this package (or any other computational physics package for that matter), is that exact computation will quickly be infeasible. Truncations introduce a necessary trade-off between computational cost and accuracy.

### TODO: Runtime vs accuracy plot